# Vapor Liquid Liquid Equilibrium (VLLE)

This notebook exemplifies the three phase equilibria calculation. The VLLE calculation is separated into two cases:

- Binary mixtures: due to degrees of freedom restrictions the VLLE is computed at given temperature or pressure. This is solved using the  ``vlleb`` function.

- Mixtures with three or more components: the VLLE is computed at given global composition, temperature and pressure. This is solved using the ``vlle`` function.


To start, the required functions are imported.

In [26]:
import numpy as np
from phasepy import component, mixture, virialgamma
from phasepy.equilibrium import vlleb, vlle

## Binary (two component) mixture VLLE

The VLLE computation for binary mixtures is solution is based on the following objective function:


$$ K_{ik} x_{ir} - x_{ik} = 0  \qquad i = 1,...,c \quad k = 2,3 $$
$$ \sum_{i=1}^c x_{ik} = 1 \qquad k = 1, 2, 3$$

Where, $x_{ik}$ is the molar fraction of the component $i$ on the phase $k$ and $ K_{ik} = x_{ik}/x_{ir} = \hat{\phi}_{ir}/\hat{\phi}_{ik} $ is the constant equilibrium respect to a reference phase $r$. 


**note:** this calculation does not check for the stability of the phases.


In the following code block, the VLLE calculation for the binary mixture of water and mtbe is exemplified.   For binary mixtures, the VLLE is computed at either given pressure (``P``) or temperature (``T``). The function ``vlleb`` requires either of those and initial guesses unknown variables (phase compositions and unknown specification).

First, the mixture and its interaction parameters are set up.

In [27]:
water = component(name='water', Tc=647.13, Pc=220.55, Zc=0.229, Vc=55.948, w=0.344861,
                  Ant=[11.64785144, 3797.41566067, -46.77830444],
                  GC={'H2O':1})

mtbe = component(name='mtbe', Tc=497.1, Pc=34.3, Zc=0.273, Vc=329.0, w=0.266059,
                 Ant=[9.16238246, 2541.97883529, -50.40534341],
                 GC={'CH3':3, 'CH3O':1, 'C':1})

mix = mixture(water, mtbe)
mix.unifac()
eos = virialgamma(mix, actmodel='unifac')

The binary VLLE at constant pressure is computed as follows:

In [28]:
P = 1.01 # bar
T0 = 320.0 # K
x0 = np.array([0.01, 0.99])
w0 = np.array([0.99, 0.01])
y0 = (x0 + w0)/2
vlleb(x0, w0, y0, T0, P, 'P', eos)

(array([0.06315848, 0.93684152]),
 array([0.99776573, 0.00223427]),
 array([0.13658506, 0.86341494]),
 array([325.34019061]))

Similarly, the binary VLLE at constant temperature is computed below:

In [29]:
P0 = 1.01 # bar
T = 320.0 # K
x0 = np.array([0.01, 0.99])
w0 = np.array([0.99, 0.01])
y0 = (x0 + w0)/2
vlleb(x0, w0, y0, P0, T, 'T', eos)

(array([0.05876527, 0.94123473]),
 array([0.99774233, 0.00225767]),
 array([0.12661976, 0.87338024]),
 array([0.83404552]))

## Multicomponent mixture VLLE

Phase stability plays a key role during equilibrium computation when dealing with more than two liquid phases. For this purpose the following modified multiphase Rachford-Rice mass balance has been proposed [Gupta et al.](https://www.sciencedirect.com/science/article/pii/037838129180021M):


$$ \sum_{i=1}^c \frac{z_i (K_{ik} \exp{\theta_k}-1)}{1+ \sum\limits^{\pi}_{\substack{j=1 \\ j \neq r}}{\psi_j (K_{ij}} \exp{\theta_j} -1)} = 0 \qquad k = 1,..., \pi,  k \neq r $$

Subject to:

$$ \psi_k \theta_k = 0 $$

In this system of equations, $z_i$ represents the global composition of the component $i$,  $ K_{ij} = x_{ij}/x_{ir} = \hat{\phi}_{ir}/\hat{\phi}_{ij} $ is the constant equilibrium of component $i$ in phase $j$ respect to the reference phase $r$, and $\psi_j$ and $\theta_j$ are the phase fraction and stability variable of the phase $j$.  

The solution strategy is similar to the classic isothermal isobaric two-phase flash. First, a reference phase must be selected, this phase is considered stable during the procedure. In an inner loop, the system of equations is solved using multidimensional Newton's method for phase fractions and stability variables and then compositions are updated in an outer loop using accelerated successive substitution (ASS).  Once the algorithm has converged, the stability variable gives information about the phase. If it takes a value of zero the phase is stable and if it is positive the phase is not.  The proposed successive substitution method can be slow, if that is the case the algorithm attempts to minimize Gibbs Free energy of the system. This procedure also ensures stable solutions and is solved using SciPy's functions.

$$ min \, {G} = \sum_{k=1}^\pi \sum_{i=1}^c F_{ik} \ln \hat{f}_{ik}  $$


The next code block exemplifies the VLLE calculation for the mixture of water, ethanol and MTBE. This is done with the ``vlle`` function, which incorporates the algorithm described above. This functions requires the global composition (``z``), temperature (``T``) and pressure (``P``). Additionally, the ``vlle`` function requires initial guesses for the composition of the phases.

In [30]:
ethanol = component(name='ethanol', Tc=514.0, Pc=61.37, Zc=0.241, Vc=168.0, w=0.643558,
                    Ant=[11.61809279, 3423.0259436, -56.48094263],
                    GC={'CH3':1, 'CH2':1, 'OH(P)':1})

mix = mixture(water, mtbe)
mix.add_component(ethanol)
mix.unifac()
eos = virialgamma(mix, actmodel='unifac')

Once the ternary mixture has been set up, the VLLE computation is performed as follows:

In [31]:
P = 1.01 # bar
T = 326. # K
Z = np.array([0.4, 0.5, 0.1])
x0 = np.array([0.95, 0.025, 0.025])
w0 = np.array([0.1, 0.7, 0.2])
y0 = np.array([0.15, 0.8, 0.05])
vlle(x0, w0, y0, Z, T, P, eos, K_tol=1e-11, full_output=True)

           T: 326.0
           P: 1.01
 error_outer: 9.450717600544585e-12
 error_inner: 1.4900494260056766e-10
        iter: 15
        beta: array([0.36005821, 0.54431549, 0.0956263 ])
       tetha: array([0., 0., 0.])
           X: array([[0.92911972, 0.00500844, 0.06587183],
       [0.0967979 , 0.77719501, 0.12600709],
       [0.13358479, 0.7859489 , 0.08046631]])
           v: [None, None, None]
      states: ['L', 'L', 'V']

For further information please also check [official documentation](https://phasepy.readthedocs.io/), or just try:

```function?```